In [ ]:
# Idea de este cuaderno

# Dado un perfil de variables de decisión, evalúe la FSM de SolarMED
# Guarde resultados en results/episode_{id}
# En attachments/FSM genere una figura para cada FSM iteration_{X}_{fsm}.svg
# En attachments/facility_diagram genere una figura para la instalación reflejando el modo de operación como iterations_{X}_facility.svg

In [17]:
from typing import Literal, Annotated
import time
import transitions as tr 
from loguru import logger
import pandas as pd
from pathlib import Path
from enum import Enum

from solarMED_optimization.fsm import SolarMED
from solarMED_optimization import SF_TS_State, MedState
from solarMED_optimization.visualization import med_relative_cons_plot, costs_plot, state_evolution_plot
from solarMED_optimization.fsm.utils import test_profile
from phd_visualizations import save_figure

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
episode_id = 'test'

report_path: Path = Path('docs/results/episode_reports') / episode_id
report_path.mkdir(parents=True, exist_ok=True) # Create output path if it doesn't exist

attachments_path: Path = report_path/ 'attachments'
attachments_path.mkdir(parents=True, exist_ok=True) # Create output path if it doesn't exist

# Generate a case study report

In [34]:
# Test case

model = SolarMED()

test_profile(model=model, attachments_path=attachments_path, n_of_steps = 3, episode_id="test2")

2024-04-15 16:08:45.196 | DEBUG    | solarMED_optimization.fsm:inform_exit_state:63 - Left state OFF
2024-04-15 16:08:45.197 | INFO     | solarMED_optimization.fsm:set_vacuum_start:352 - Started generating vacuum, it will take 5 samples to complete
2024-04-15 16:08:45.197 | DEBUG    | solarMED_optimization.fsm:inform_enter_state:59 - Entered state GENERATING_VACUUM
2024-04-15 16:10:19.754 | INFO     | solarMED_optimization.fsm:is_vacuum_done:416 - Still generating vacuum, 1/5 samples completed
2024-04-15 16:10:19.754 | DEBUG    | solarMED_optimization.fsm:inform_exit_state:63 - Left state GENERATING_VACUUM
2024-04-15 16:10:19.755 | INFO     | solarMED_optimization.fsm:reset_fsm:338 - Resetting MED_FSM FSM
2024-04-15 16:10:19.756 | INFO     | solarMED_optimization.fsm:set_vacuum_reset:356 - Cancelled vacuum generation
2024-04-15 16:10:19.756 | DEBUG    | solarMED_optimization.fsm:inform_enter_state:59 - Entered state OFF
2024-04-15 16:10:19.756 | DEBUG    | solarMED_optimization.fsm.uti

KeyboardInterrupt: 

In [29]:
# Generate plots

# State evolution
fig = state_evolution_plot(df)
save_figure(fig, 'state_evolution', attachments_path, formats=['svg'])

# Costs plot
fig = costs_plot(df)
save_figure(fig, 'costs_plot', attachments_path, formats=['svg'])

# Inputs evolution
# TODO: Implementar

# save_figure(fig, )

ValueError: Column 'sf_ts_state' not found in the DataFrame

In [31]:

def generate_iteration_slide_str(iteration_idx: int, df: pd.DataFrame):
    iteration_str = f""" 

<grid drag="70 50" drop="topleft" bg="red">
Facility diagram

![{df.iloc[iteration_idx]['state']}](../../operating_modes/{df.iloc[iteration_idx]['state']}.svg)
</grid>

<grid drop="right" drag="30 100" bg="green">
Finite state machine
![Finite state machine](./attachments/iteration_{iteration_idx}_{episode_id}_fsm.svg)
</grid>

<grid drag="35 50" drop="bottomleft" bg="coral">
Costs
![Costs](./attachments/iteration_{iteration_idx}_costs.svg)
</grid>

<grid drag="35 50" drop="35 50" bg="blue">
MED relative consumptions
![MED relative consumptions](./attachments/iteration_{iteration_idx}_med_relative_cons.svg)
</grid>

---
"""
    
    return iteration_str


report_str = f"""


# Solar MED 
{episode_id} results

---

## State evolution

---

<grid drag="100 40" drop="top" bg="red">
**State evolution**

![State evolution](./attachments/state_evolution.svg)
</grid>


<grid drag="100 40" drop="bottom" bg="blue">
Decision variables
![Decision variables evolution](./attachments/decision_variables_evolution.svg)
</grid>

---

## Iterations

---

{''.join([generate_iteration_slide_str(i, df) for i in df.index])}

"""

In [32]:
# Save report
with open(report_path / f'report.md', 'w') as f:
    f.write(report_str)